In [1]:
import pandas as pd
import plotly.express as px 
import streamlit as st
import warnings
import numpy as np
warnings.filterwarnings('ignore')
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, HoverTool, NumeralTickFormatter
from bokeh.transform import factor_cmap
from bokeh.transform import linear_cmap
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler, OneHotEncoder
from bokeh.palettes import Spectral10
from bokeh.transform import jitter

In [2]:
df = pd.read_csv("churn.csv", encoding = "ISO-8859-1")

In [3]:
df.head()

,ExistingLost,Age,Gender,Dependent,Education,Marital,Income,CardCat,PeriodOfRelationship,TotalNumberProducts,MonthsInactive,ContactsCount,CreditLimit,TotalBal,AvgOpenToBuy,ChangeTransAmountQ4Q1,TotalTransAmt,TotalTransCountQ4Q1,ChangeTransCountQ4Q1,UtilRatio
0,Existing Customer,45,M,3,High School,Married,$60K - $80K,Blue,39,5,1,3,12691.0,777,11914.0,1.335,1144,42,1.625,0.061
1,Existing Customer,49,F,5,Graduate,Single,Less than $40K,Blue,44,6,1,2,8256.0,864,7392.0,1.541,1291,33,3.714,0.105
2,Existing Customer,51,M,3,Graduate,Married,$80K - $120K,Blue,36,4,1,0,3418.0,0,3418.0,2.594,1887,20,2.333,0.000
3,Existing Customer,40,M,3,Uneducated,Married,$60K - $80K,Blue,21,5,1,0,4716.0,0,4716.0,2.175,816,28,2.500,0.000
4,Existing Customer,44,M,2,Graduate,Married,$40K - $60K,Blue,36,3,1,2,4010.0,1247,2763.0,1.376,1088,24,0.846,0.311


In [4]:
df.describe()

,Age,Dependent,PeriodOfRelationship,TotalNumberProducts,MonthsInactive,ContactsCount,CreditLimit,TotalBal,AvgOpenToBuy,ChangeTransAmountQ4Q1,TotalTransAmt,TotalTransCountQ4Q1,ChangeTransCountQ4Q1,UtilRatio
count,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000,7081.000000
mean,46.347691,2.337805,35.981359,3.819376,2.342607,2.454456,8492.773831,1167.501624,7325.272207,0.760584,4394.299816,64.503319,0.711508,0.282313
std,8.041225,1.291649,8.002609,1.544444,0.995104,1.104917,9126.072520,812.315606,9131.217585,0.223139,3468.461606,23.809330,0.238693,0.278731
min,26.000000,0.000000,13.000000,1.000000,0.000000,0.000000,1438.300000,0.000000,3.000000,0.000000,510.000000,10.000000,0.000000,0.000000
25%,41.000000,1.000000,31.000000,3.000000,2.000000,2.000000,2498.000000,463.000000,1248.000000,0.629000,2089.000000,44.000000,0.583000,0.026000
50%,46.000000,2.000000,36.000000,4.000000,2.000000,2.000000,4287.000000,1282.000000,3250.000000,0.735000,3831.000000,67.000000,0.700000,0.186000
75%,52.000000,3.000000,40.000000,5.000000,3.000000,3.000000,10729.000000,1781.000000,9491.000000,0.858000,4740.000000,80.000000,0.818000,0.515000
max,73.000000,5.000000,56.000000,6.000000,6.000000,6.000000,34516.000000,2517.000000,34516.000000,3.397000,17995.000000,134.000000,3.714000,0.999000


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7081 entries, 0 to 7080
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ExistingLost           7081 non-null   object 
 1   Age                    7081 non-null   int64  
 2   Gender                 7081 non-null   object 
 3   Dependent              7081 non-null   int64  
 4   Education              7081 non-null   object 
 5   Marital                7081 non-null   object 
 6   Income                 7081 non-null   object 
 7   CardCat                7081 non-null   object 
 8   PeriodOfRelationship   7081 non-null   int64  
 9   TotalNumberProducts    7081 non-null   int64  
 10  MonthsInactive         7081 non-null   int64  
 11  ContactsCount          7081 non-null   int64  
 12  CreditLimit            7081 non-null   float64
 13  TotalBal               7081 non-null   int64  
 14  AvgOpenToBuy           7081 non-null   float64
 15  Chan

In [6]:
st.set_page_config(page_title = "Churning customers", page_icon = ":credit_card:", layout = "wide")

In [7]:
st.title(":credit_card: Credit Card Churning Customers Analysis")
st.markdown('<style>div.block-container{padding-top:1rem;}</style>', unsafe_allow_html=True)

2023-11-21 20:04:29.291 
  command:

    streamlit run /Users/annageorgieva/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


DeltaGenerator()

In [16]:
# Custom CSS to inject for changing the selection color of Streamlit widgets
st.markdown("""
<style>
/* CSS selector for the checkbox color */
input[type="checkbox"]:checked + label:before {
    background-color: #5E18EB !important;
    border-color: #5E18EB !important;
}

/* CSS selector for the background color of selected options in multiselect */
.css-12jo7m5, .css-y2l8u7 {
    background-color: #5E18EB !important;
}

/* CSS selector for the multiselect dropdown styling */
.css-1okebmr-indicatorSeparator {
    background-color: transparent !important;
}

/* CSS selector for the multiselect clear and dropdown indicators */
.css-tlfecz-indicatorContainer, .css-1gtu0rj-indicatorContainer {
    color: #5E18EB !important;
}

/* CSS selectors for the slider thumb color */
.css-1s4pdt5-StyledSlider, .css-1s4pdt5-StyledSlider:hover {
    background-color: #5E18EB !important;
}

/* CSS selectors for the slider track color */
.css-rmt63o-StyledSliderTrack, .css-rmt63o-StyledSliderTrack:hover {
    background-color: #5E18EB !important;
}

</style>
""", unsafe_allow_html=True)

DeltaGenerator()

In [9]:
fl = st.file_uploader(":file_folder: Upload a file", type=(["csv", "txt", "xlsx", "xls"]))
if fl is not None:
    filename = fl.name
    st.write(filename)
    df = pd.read_csv(filename, encoding = "ISO-8859-1")
else:
    df = pd.read_csv("churn.csv", encoding = "ISO-8859-1")

In [17]:
# Title for the sidebar
st.sidebar.header('Choose your filter:')

# Define categories and corresponding colors
categories = ['Attrited Customer', 'Existing Customer']
category_colors = ['#5E18EB', '#A579FF']
color_mapping = {'Attrited Customer': category_colors[0], 'Existing Customer': category_colors[1]}

# List of all columns for which you want to create filters
all_columns = ['Gender', 'Education', 'Marital', 'Income', 'CardCat', 'MonthsInactive',
               'Age', 'Dependent', 'PeriodOfRelationship', 'TotalNumberProducts',
               'ContactsCount', 'CreditLimit', 'TotalBal', 'AvgOpenToBuy',
               'ChangeTransAmountQ4Q1', 'TotalTransAmt', 'TotalTransCountQ4Q1',
               'ChangeTransCountQ4Q1', 'UtilRatio']

# Create filters in the sidebar
selected_filters = {}
for col in all_columns:
    if df[col].dtype == 'object':
        # Use multiselect for categorical columns
        selected_filters[col] = st.sidebar.multiselect(f'Select {col}', df[col].unique())
    else:
        # Use sliders for numerical columns
        min_val = df[col].min()
        max_val = df[col].max()
        selected_filters[col] = st.sidebar.slider(f'Select range for {col}', min_val, max_val, (min_val, max_val))

# Let the user select a column to plot
chosen_col = st.sidebar.selectbox('Select a column to plot', all_columns)

# Filtering the dataframe based on the selections
filtered_df = df.copy()
for col, values in selected_filters.items():
    if isinstance(values, list) and values:  # For categorical filters
        filtered_df = filtered_df[filtered_df[col].isin(values)]
    elif isinstance(values, tuple):  # For numerical filters
        filtered_df = filtered_df[filtered_df[col].between(values[0], values[1])]

# Create a container for the plot
plot_container = st.container()

with plot_container:
    # Custom HTML for the title with larger font size and bold text
    st.markdown("""
    <style>
    .title {
        font-size: small; 
        font-weight: bold;
    }
    </style>
    <div class="title">Customize Your Analysis: Select Variables to Compare Results</div>
    """, unsafe_allow_html=True)    

    # Create and display the count plot if a column is selected
    if chosen_col:
        fig = px.histogram(filtered_df, x=chosen_col, color='ExistingLost',
                           color_discrete_map=color_mapping,
                           title=f'Count of ExistingLost for {chosen_col}')
        fig.update_layout(title_text='')
        st.plotly_chart(fig)
    else:
        st.write("Please select a column for plotting.")

In [11]:
# Convert 'ExistingLost' to a numeric value: for example, 'Existing Customer' to 1, and 'Lost' to 0
df['CurrentChurned'] = df['ExistingLost'].apply(lambda x: 1 if x == 'Existing Customer' else 0)

# Now include this new numeric column in your correlation matrix calculation
corr_matrix = df[['CurrentChurned', 'Age', 'PeriodOfRelationship', 'TotalNumberProducts', 'MonthsInactive', 
                  'ContactsCount', 'CreditLimit', 'TotalBal', 'AvgOpenToBuy', 
                  'ChangeTransAmountQ4Q1', 'TotalTransAmt', 'TotalTransCountQ4Q1', 
                  'ChangeTransCountQ4Q1', 'UtilRatio']].corr()

# Custom color scale
color_scale = [
    [0, '#A579FF'],  
    [1, '#5E18EB']   
]

# Generate the heatmap
fig = px.imshow(corr_matrix, 
                x=corr_matrix.columns, 
                y=corr_matrix.columns, 
                color_continuous_scale=color_scale,
                text_auto=True, 
                aspect="auto")

fig.update_layout(coloraxis_colorbar=dict(
    title='Correlation',
    tickvals=[-1, 1], 
    ticktext=['Minimum', 'Maximum'],
    lenmode='fraction', 
    len=1
))

# Create columns with a custom layout, including 'spacer' columns
col2, spacer, col3 = st.columns([3, 1.05, 3])

with col2:
    # Custom HTML for the title with larger font size and bold text
    st.markdown("""
    <style>
    .title {
        font-size: small; 
        font-weight: bold;
    }
    </style>
    <div class="title">Correlation Matrix of Credit Card Customer Attributes</div>
    """, unsafe_allow_html=True)
    
    st.plotly_chart(fig)

In [12]:
# Convert 'MonthsInactive' to string if treating as categorical
df['MonthsInactive'] = df['MonthsInactive'].astype(str)

# Manually create a jitter effect
df['JitteredMonthsInactive'] = df['MonthsInactive'].apply(lambda x: float(x) + np.random.uniform(-0.3, 0.3))

# Create the Plotly scatter plot
fig = px.scatter(df, 
                 x='PeriodOfRelationship', 
                 y='JitteredMonthsInactive', 
                 color='ExistingLost',
                 labels={'JitteredMonthsInactive': 'Months Inactive'},
                 title="Customer Relationship Analysis",
                 color_discrete_sequence=['#A579FF', '#5E18EB'])

# Customize the hover data
fig.update_traces(hovertemplate="Churn Status: %{color}<br>Period of Relationship: %{x}<br>Months Inactive: %{y}")
fig.update_layout(title_text='')

# Customize the layout
fig.update_layout(legend_title_text='Churn Status')

with col3:
    # Custom HTML for the title with larger font size and bold text
    st.markdown("""
    <style>
    .title {
        font-size: small; 
        font-weight: bold;
    }
    </style>
    <div class="title">Customer relationship analysis</div>
    """, unsafe_allow_html=True)
    
    st.plotly_chart(fig)

In [13]:
# Define categories and corresponding colors
categories = ['Attrited Customer', 'Existing Customer']
category_colors = ['#5E18EB', '#A579FF']
color_mapping = {'Attrited Customer': category_colors[0], 'Existing Customer': category_colors[1]}

# Plot the histogram with LoanAmount on the x-axis and stacked Loan_Status
fig = px.histogram(df,
                   x="CardCat",
                   color="ExistingLost",
                   color_discrete_map=color_mapping,
                   barmode='stack',
                   labels={'CartCat': 'Card Category'},
                   text_auto=True)

# Update layout for centering the title and adjusting y-axis title
fig.update_layout(title_x=0.5, yaxis_title='Count')
fig.update_layout(title_text='')
    
with col2:

    # Custom HTML for the title of the new chart
    st.markdown("""
    <style>
    .title {
        font-size: small; 
        font-weight: bold;
    }
    </style>
    <div class="title">Stacked Distribution of Churned and Existing Customers by Card Category</div>
    """, unsafe_allow_html=True)
    
    # Display the new histogram chart
    st.plotly_chart(fig)

In [14]:
numerical=df.select_dtypes([np.number]).columns
print(numerical)
categorycol=df.select_dtypes('object').columns
print(categorycol)

Index(['Age', 'Dependent', 'PeriodOfRelationship', 'TotalNumberProducts',
       'ContactsCount', 'CreditLimit', 'TotalBal', 'AvgOpenToBuy',
       'ChangeTransAmountQ4Q1', 'TotalTransAmt', 'TotalTransCountQ4Q1',
       'ChangeTransCountQ4Q1', 'UtilRatio', 'CurrentChurned',
       'JitteredMonthsInactive'],
      dtype='object')
Index(['ExistingLost', 'Gender', 'Education', 'Marital', 'Income', 'CardCat',
       'MonthsInactive'],
      dtype='object')
